## Check syn files

In [ ]:
syn_folder = "D:/碩班 許盛軒_research/vicon dataset 2/syn"

In [5]:
import os
count = 0
for (root,dirs,files) in os.walk(syn_folder, topdown=True): 
        if 'syn_gt.csv' in files and 'syn_imu.csv' in files:
            count+=1
            print (root.replace(syn_folder+'\\',''))
print(f"count: {count}")

hand_8\fast_cy_1_arm
hand_8\fast_cy_2_arm
hand_8\fast_cy_3_arm
hand_8\fast_cy_4_arm
hand_8\fast_sh_1_arm
hand_8\fast_sh_2_arm
hand_8\fast_sh_3_arm
hand_8\fast_sh_4_arm
hand_8\fast_yj_2_arm
hand_8\fast_yj_3_arm
hand_8\fast_yj_4_arm
hand_8\mid_cy_1_arm
hand_8\mid_cy_2_arm
hand_8\mid_cy_3_arm
hand_8\mid_cy_4_arm
hand_8\mid_sh_1_arm
hand_8\mid_sh_2_arm
hand_8\mid_sh_3_arm
hand_8\mid_sh_4_arm
hand_8\mid_yj_1_arm
hand_8\mid_yj_2_arm
hand_8\mid_yj_3_arm
hand_8\mid_yj_4_arm
hand_8\slow_cy_1_arm
hand_8\slow_cy_2_arm
hand_8\slow_cy_3_arm
hand_8\slow_cy_4_arm
hand_8\slow_sh_1_arm
hand_8\slow_sh_2_arm
hand_8\slow_sh_3_arm
hand_8\slow_sh_4_arm
hand_8\slow_yj_1_arm
hand_8\slow_yj_2_arm
hand_8\slow_yj_3_arm
hand_8\slow_yj_4_arm
hand_front\fast_cy_1_arm
hand_front\fast_cy_2_arm
hand_front\fast_cy_3_arm
hand_front\fast_cy_4_arm
hand_front\fast_sh_1_arm
hand_front\fast_sh_2_arm
hand_front\fast_sh_3_arm
hand_front\fast_sh_4_arm
hand_front\fast_yj_1_arm
hand_front\fast_yj_2_arm
hand_front\fast_yj_3_arm
ha

## Pack train dataset to NPZ file

In [11]:
from Tracking3D_dataset import load_dataset_9d_quat_2, force_quaternion_uniqueness, load_hsh_datasets
from Tracking3D_util import generate_trajectory_6d_quat
import plotly.graph_objs as go
import joblib
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

train_list_file = "D:/碩班 許盛軒_research/vicon dataset 2/Preprocessing/train_list.txt"
train_file_dest = "D:/碩班 許盛軒_research/vicon dataset 2/npz/"
with open(train_list_file) as f:
    lines = f.read().splitlines()

window_size = 100
stride = 5

x_gyro = []
x_acc = []
x_mag = []
gyro_test = []
acc_test = []
mag_test = []
y_delta_p = []
y_delta_q = []
y_delta_p_test = []
y_delta_q_test = []

for i, line in enumerate(lines):
            #-------------------------------------------------9 dof----------------------------------------------------------
            print('loading '+line+'...')
            cur_imu_data_filename =  syn_folder + '/' +  line + "/syn_imu.csv"
            cur_gt_data_filename =  syn_folder + '/' +  line + "/syn_gt.csv"
            cur_gyro_data, cur_acc_data, cur_mag_data, cur_pos_data, cur_ori_data = load_hsh_datasets(cur_imu_data_filename, cur_gt_data_filename)

            [cur_x_gyro, cur_x_acc, cur_x_mag], [cur_y_delta_p, cur_y_delta_q], init_p, init_q = load_dataset_9d_quat_2(cur_gyro_data, cur_acc_data, cur_mag_data, cur_pos_data, cur_ori_data, window_size, stride)
            cur_x_gyro, s_gyro_test, cur_y_delta_p, s_y_delta_p_test, cur_x_acc, s_acc_test, cur_y_delta_q, s_y_delta_q_test, cur_x_mag, s_mag_test = train_test_split(cur_x_gyro,cur_y_delta_p,cur_x_acc,cur_y_delta_q, cur_x_mag,test_size=0.3)
            x_gyro.append(cur_x_gyro)
            x_acc.append(cur_x_acc)
            x_mag.append(cur_x_mag)
            y_delta_p.append(cur_y_delta_p)
            y_delta_q.append(cur_y_delta_q)
            gyro_test.append(s_gyro_test)
            acc_test.append(s_acc_test)
            mag_test.append(s_mag_test)
            y_delta_p_test.append(s_y_delta_p_test)
            y_delta_q_test.append(s_y_delta_q_test)

x_gyro = np.vstack(x_gyro)
x_acc = np.vstack(x_acc)
y_delta_p = np.vstack(y_delta_p)
y_delta_q = np.vstack(y_delta_q)
 
gyro_test = np.vstack(gyro_test)
acc_test = np.vstack(acc_test)
y_delta_p_test = np.vstack(y_delta_p_test)
y_delta_q_test = np.vstack(y_delta_q_test)
 
#----------mag-------------
x_mag = np.vstack(x_mag)
mag_test = np.vstack(mag_test)
x_gyro, x_acc, x_mag, y_delta_p, y_delta_q = shuffle(x_gyro, x_acc, x_mag, y_delta_p, y_delta_q)
gyro_test, acc_test, mag_test, y_delta_p_test, y_delta_q_test = shuffle(gyro_test, acc_test, mag_test, y_delta_p_test, y_delta_q_test)
#--------------------------
 
 
# Normalize data
print("normalizing...")
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(x_acc.reshape(-1, x_acc.shape[-1]))
x_acc = scaler.transform(x_acc.reshape(-1, x_acc.shape[-1])).reshape(x_acc.shape)
acc_test = scaler.transform(acc_test.reshape(-1, acc_test.shape[-1])).reshape(acc_test.shape)
joblib.dump(scaler, train_file_dest+'acc_minmax.pkl')

scaler.fit(x_gyro.reshape(-1, x_gyro.shape[-1]))
x_gyro = scaler.transform(x_gyro.reshape(-1, x_gyro.shape[-1])).reshape(x_gyro.shape)
gyro_test = scaler.transform(gyro_test.reshape(-1, gyro_test.shape[-1])).reshape(gyro_test.shape)
joblib.dump(scaler, train_file_dest+'gyro_minmax.pkl')

scaler.fit(x_mag.reshape(-1, x_mag.shape[-1]))
x_mag = scaler.transform(x_mag.reshape(-1, x_mag.shape[-1])).reshape(x_mag.shape)
mag_test = scaler.transform(mag_test.reshape(-1, mag_test.shape[-1])).reshape(mag_test.shape)
joblib.dump(scaler, train_file_dest+'mag_minmax.pkl')


# save npz file
print("saving...")
np.savez(train_file_dest+"train.npz",x_gyro=x_gyro, x_acc=x_acc, x_mag=x_mag, y_delta_p=y_delta_p, y_delta_q=y_delta_q, init_p=init_p, init_q=init_q, acc_test=acc_test, gyro_test=gyro_test, mag_test=mag_test,  y_delta_p_test=y_delta_p_test, y_delta_q_test=y_delta_q_test)

loading hand_8\fast_cy_1_arm...
loading hand_8\fast_cy_2_arm...
loading hand_8\fast_cy_3_arm...
loading hand_8\fast_cy_4_arm...
loading hand_8\fast_sh_1_arm...
loading hand_8\fast_sh_3_arm...
loading hand_8\fast_sh_4_arm...
loading hand_8\fast_yj_2_arm...
loading hand_8\fast_yj_3_arm...
loading hand_8\fast_yj_4_arm...
loading hand_8\mid_cy_1_arm...
loading hand_8\mid_cy_2_arm...
loading hand_8\mid_cy_3_arm...
loading hand_8\mid_sh_1_arm...
loading hand_8\mid_sh_2_arm...
loading hand_8\mid_sh_3_arm...
loading hand_8\mid_sh_4_arm...
loading hand_8\mid_yj_1_arm...
loading hand_8\mid_yj_2_arm...
loading hand_8\mid_yj_3_arm...
loading hand_8\mid_yj_4_arm...
loading hand_8\slow_cy_1_arm...
loading hand_8\slow_cy_2_arm...
loading hand_8\slow_cy_3_arm...
loading hand_8\slow_cy_4_arm...
loading hand_8\slow_sh_1_arm...
loading hand_8\slow_sh_2_arm...
loading hand_8\slow_sh_3_arm...
loading hand_8\slow_sh_4_arm...
loading hand_8\slow_yj_1_arm...
loading hand_8\slow_yj_2_arm...
loading hand_8\slow

## Pack testing data to NPZ files

In [10]:
test_list_file = "D:/碩班 許盛軒_research/vicon dataset 2/Preprocessing/test_list.txt"
train_file_dest = "D:/碩班 許盛軒_research/vicon dataset 2/npz/"
with open(test_list_file) as f:
    lines = f.read().splitlines()

for i, line in enumerate(lines):
    print('loading '+line+'...')
    cur_imu_data_filename =  syn_folder + '/' +  line + "/syn_imu.csv"
    cur_gt_data_filename =  syn_folder + '/' +  line + "/syn_gt.csv"
    gyro_data, acc_data, mag_data, pos_data, ori_data = load_hsh_datasets(cur_imu_data_filename, cur_gt_data_filename)
    [x_gyro, x_acc, x_mag], [y_delta_p, y_delta_q], init_p, init_q = load_dataset_9d_quat_2(gyro_data, acc_data, mag_data, pos_data, ori_data, window_size, stride)

    
    # Normalize data
    print("normalizing...")
    scaler = joblib.load(train_file_dest+'acc_minmax.pkl')        
    x_acc = scaler.transform(x_acc.reshape(-1, x_acc.shape[-1])).reshape(x_acc.shape)                
    scaler = joblib.load(train_file_dest+'gyro_minmax.pkl')        
    x_gyro = scaler.transform(x_gyro.reshape(-1, x_gyro.shape[-1])).reshape(x_gyro.shape)                
    scaler = joblib.load(train_file_dest+'mag_minmax.pkl')
    x_mag = scaler.transform(x_mag.reshape(-1, x_mag.shape[-1])).reshape(x_mag.shape)


    # save npz file
    print(f"saving {train_file_dest}test/{i}.npz ...")
    np.savez(train_file_dest+'test/'+str(i)+'.npz',x_gyro=x_gyro, x_acc=x_acc, x_mag=x_mag, y_delta_p=y_delta_p, y_delta_q=y_delta_q, init_p=init_p, init_q=init_q)

loading hand_8\fast_sh_2_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/0.npz ...
loading hand_8\mid_cy_4_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/1.npz ...
loading hand_8\slow_yj_3_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/2.npz ...
loading hand_front\fast_cy_3_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/3.npz ...
loading hand_front\mid_cy_3_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/4.npz ...
loading hand_front\slow_cy_3_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/5.npz ...
loading hand_wave\fast_yj_4_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/6.npz ...
loading hand_wave\mid_sh_3_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/7.npz ...
loading hand_wave\slow_sh_1_arm...
normalizing...
saving D:/碩班 許盛軒_research/vicon dataset 2/npz/test/8.npz ...
loading wa